# Using Elasticsearch as Backend
Our pipeline remains very similar to the one in [Tutorial 1](https://github.com/deepset-ai/haystack/blob/master/tutorials/Tutorial1_Basic_QA_Pipeline.ipynb), where we had a SQL backend.
We therefore only highlight the three key differences here.

In [1]:
from haystack import Finder
from haystack.database.elasticsearch import ElasticsearchDocumentStore

from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from haystack.reader.transformers import TransformersReader
from haystack.retriever.elasticsearch import ElasticsearchRetriever
from haystack.utils import print_answers

In [2]:
# Get documents (same as in Tutorial 1)
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

03/17/2020 12:54:42 - INFO - haystack.indexing.io -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1167348/1167348 [00:00<00:00, 4908001.41B/s]


True

### Difference 1: Initialize a document store for Elasticsearch
This requires a running Elasticsearch instance. To run one locally, you can execute:

```docker run -d -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" elasticsearch:7.5.1```


In [4]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

03/17/2020 12:55:08 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.971s]


### Difference 2: Split docs into paragraphs before indexing
Set `split_paragraphs=True`

In [5]:
write_documents_to_db(
    document_store=document_store,
    document_dir=doc_dir,
    clean_func=clean_wiki_text,
    only_empty_db=True,
    split_paragraphs=True
)

03/17/2020 12:55:16 - INFO - elasticsearch -   POST http://localhost:9200/_count [status:200 request:0.221s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.232s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.054s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.029s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.029s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.046s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.024s]
03/17/2020 12:55:17 - INFO - elasticsearch -   POST http://localhost:9200/docum

03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.024s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.029s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.024s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.029s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.029s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:55:19 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.026s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.023s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.042s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.021s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.026s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:21 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:55:23 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.021s]
03/17/2020 12:55:23 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.509s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.093s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.055s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.068s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.072s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.073s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.073s]
03/17/2020 12:55:24 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.022s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.017s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.017s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.024s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.022s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.023s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.022s]
03/17/2020 12:55:26 - INFO - elasticsearch -   POST http://localhost:920

...

03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.016s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.016s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.020s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.014s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.013s]
03/17/2020 12:56:15 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.014s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.029s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.024s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.024s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.026s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.022s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.014s]
03/17/2020 12:56:16 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:56:18 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.018s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.016s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.017s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.013s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.012s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.010s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.014s]
03/17/2020 12:56:19 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.016s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.048s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.012s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.020s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.017s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.023s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.021s]
03/17/2020 12:56:20 - INFO - elasticsearch -   POST http://localhost:920

03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.030s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.019s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.016s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.023s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.016s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.015s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.021s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_doc [status:201 request:0.025s]
03/17/2020 12:56:21 - INFO - elasticsearch -   POST http://localhost:920

### Difference 3: Use the native Elasticsearch implementation of BM25 as a retriever

In [6]:
retriever = ElasticsearchRetriever(document_store=document_store)

### Init Reader & and use Finder to get answer (same as in Tutorial 1)

In [7]:
reader = TransformersReader(
    model="deepset/bert-base-cased-squad2",
    tokenizer="deepset/bert-base-cased-squad2",
    use_gpu=-1
)
finder = Finder(reader, retriever)

03/17/2020 12:56:32 - INFO - filelock -   Lock 5035474056 acquired on /Users/bogdan/.cache/torch/transformers/63685c11f75a43d37a5167ef7efb7cb5d0851ae5e54a6dac3593974efce34326.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock


03/17/2020 12:56:33 - INFO - filelock -   Lock 5035474056 released on /Users/bogdan/.cache/torch/transformers/63685c11f75a43d37a5167ef7efb7cb5d0851ae5e54a6dac3593974efce34326.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock


In [8]:
prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)

03/17/2020 12:56:42 - INFO - elasticsearch -   GET http://localhost:9200/document/_search [status:200 request:1.194s]
03/17/2020 12:56:42 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever: [{'paragraph_id': 'kjhZ6HABGsS8JiHo6Apl', 'document_id': 2295, 'document_name': '450_Baelor.txt'}, {'paragraph_id': '0zhZ6HABGsS8JiHoOwJc', 'document_id': 312, 'document_name': '224_The_Night_Lands.txt'}, {'paragraph_id': 'vjhZ6HABGsS8JiHoTgPC', 'document_id': 547, 'document_name': '332_Sansa_Stark.txt'}, {'paragraph_id': 'pThZ6HABGsS8JiHo_wsu', 'document_id': 2570, 'document_name': '191_Gendry.txt'}, {'paragraph_id': 'lThZ6HABGsS8JiHo6Aql', 'document_id': 2298, 'document_name': '450_Baelor.txt'}, {'paragraph_id': 'NzhZ6HABGsS8JiHo4Qrc', 'document_id': 2204, 'document_name': '43_Arya_Stark.txt'}, {'paragraph_id': '7jhZ6HABGsS8JiHo2wnI', 'document_id': 2131, 'document_name': '487_Ramsay_Bolton.txt'}, {'paragraph_id': 'YTha6HABGsS8JiHoEAx3', 'document_id': 2758, 'document_n

In [9]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Balon',
        'context': " captivity, as Theon's father Balon has recently died and "
                   "Theon's"},
    {   'answer': 'Eddard,',
        'context': " She travels with her father, Eddard, to King's Landing "
                   'when he is '},
    {   'answer': "Robert's",
        'context': "rdered that all of his father Robert's bastards be killed, "
                   'but Yoren'},
    {   'answer': 'Lord Eddard Stark,',
        'context': 'the Hand to honour her father Lord Eddard Stark, Sansa '
                   'Stark is enchanted by t'},
    {   'answer': 'Ned',
        'context': '=\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King'"}]
